<a href="https://colab.research.google.com/github/FaisalRashid9775/AI_Agents/blob/main/%5B4%5DTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tools**

In earlier we talk about agent have LLM and Tools. LLM do not have real time data. For giving it real time details we provide it tool. Tool is just a python function.


Tool calling is when AI decide to use a tool.
When LLM have not data it check its tool box and call appropriate tool.


We just do two things.

1. Make a function with function_tool
2. Add tool into agent

agent = Agent(name = '',instructions = '',model = '', tools = [tools name])

tools take strings so we can add more than 1 tools

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 25.9 MB/s eta 0:00:00


In [ ]:
from agents import Runner, Agent, AsyncOpenAI,set_tracing_disabled,OpenAIChatCompletionsModel,function_tool
from agents.run import RunConfig
from google.colab import userdata
api_key= userdata.get('GOOGLE_API_KEY')

import nest_asyncio
nest_asyncio.apply()

client : AsyncOpenAI = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-2.5-flash',
    openai_client=client
)

set_tracing_disabled(disabled= True)
#function_tool decorator convert normal function into a Tool.
@function_tool()
def add (a:int,b:int) -> int:
  print('Tool Add Calling')
  return a+b
math_agent = Agent(
    name='Math-agent',
    instructions='You are helpful math agent',
    model=model,
    tools = [add]  # Adding Tool to a Agent
    )
result = Runner.run_sync(math_agent,'what is 2+2')
print(result.final_output)

Tool Add Calling
The sum of 2 and 2 is 4.


# **Tavily Search Tool**

In [ ]:
!pip install -Uq tavily-python

In [ ]:
from agents import Runner, Agent, AsyncOpenAI,set_tracing_disabled,OpenAIChatCompletionsModel,function_tool
from agents.run import RunConfig
from google.colab import userdata
from tavily import TavilyClient
api_key= userdata.get('GOOGLE_API_KEY')
tavily_api_key = userdata.get('TAVILY_API_KEY')
tavily_client = TavilyClient(api_key=tavily_api_key)

import nest_asyncio
nest_asyncio.apply()

client : AsyncOpenAI = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-2.5-flash',
    openai_client=client
)
set_tracing_disabled(disabled= True)
@function_tool()
def search(query:str):
  print('search tool calling')
  response = tavily_client.search(query)
  return response

math_agent = Agent(
    name='Math-agent',
    instructions='You are helpful math agent',
    model=model,
    tools = [search]  # Adding Tool to a Agent
    )
result = Runner.run_sync(math_agent,'whats the temperature in Islamabd')
print(result.final_output)

search tool calling
The temperature in Islamabad is currently 89°F, with a RealFeel® of 78°F. The high for today is 94°F (32°C).


# **Page Extraction**

In [ ]:
def read_url(url:str)->str:
    print(f"Reading{url}")
    response =await tavily_client.extract(url)